# XGBoost

#### Import libraries

In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

#### Read data

In [ ]:
df = pd.read_csv('../../../datasets/parte2/train_prepared.csv', na_filter=False)

#### Turn categorical features into numeric

In [ ]:
replace_map = {'None':0, 'Low':1, 'Medium':2, 'High':3, 'Very High':4}

df['Injeção na rede (kWh)'] = df['Injeção na rede (kWh)'].replace(replace_map).astype(int)

#### X and y arrays

In [ ]:
X = df.drop(['Injeção na rede (kWh)'], axis=1)
y = df['Injeção na rede (kWh)']

#### Training 

##### Grid Search

Using GridSearchCV to find the best hyperparameters

In [ ]:
param_grid = {
    'max_depth': [4], 
    'min_child_weight': [1, 2, 3],  
    'subsample': [0.5, 0.6, 0.7],  
    'colsample_bytree': [0.6, 0.8, 1], 
    'learning_rate': [0.01, 0.1],  
    'n_estimators': [100],
}

model = XGBClassifier(objective='multi:softmax', random_state=2023)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, refit=True, cv=10, verbose=0, n_jobs=-1, error_score="raise") # scoring='f1'
grid_search.fit(X, y)

Inspect the best parameters

In [ ]:
grid_search.best_params_ 

In [ ]:
grid_search.best_score_ 

Get the best estimator

In [ ]:
model = grid_search.best_estimator_

Get the predictions using the trained model

In [ ]:
df_test = pd.read_csv('../../../datasets/parte2/test_prepared.csv')
predictions_test = model.predict(df_test)

In [ ]:
df_predictions = pd.DataFrame({'Result': predictions_test})
df_predictions['RowId'] = range(1, len(predictions_test) + 1)
df_predictions = df_predictions[['RowId', 'Result']]

replace_map = { 0:'None', 1:'Low', 2:'Medium', 3:'High', 4:'Very High'} 

df_predictions['Result'] = df_predictions['Result'].replace(replace_map)

df_predictions.to_csv('../../../datasets/parte2/kaggle-xgboost-sem-instant-cv20.csv', index=False)